In [ ]:
import os
import whisperx
from pydub import AudioSegment
import shutil
import torch

# Trỏ đường dẫn đến thư viện CUDA vừa cài trong venv
venv_path = os.path.join(os.getcwd(), "venv", "Lib", "site-packages", "nvidia", "cublas", "bin")
if os.path.exists(venv_path):
    os.environ["PATH"] += os.pathsep + venv_path

# 1. Chỉ định đường dẫn trực tiếp cho Pydub
ffmpeg_path = r"D:\Study\7-SP26\DATxSLP\ffmpeg.exe"
ffprobe_path = r"D:\Study\7-SP26\DATxSLP\ffprobe.exe" # Đảm bảo bạn có file này trong folder

AudioSegment.converter = ffmpeg_path
AudioSegment.ffprobe = ffprobe_path

# 2. Thêm vào môi trường hệ thống của Python
os.environ["PATH"] += os.pathsep + r"D:\Study\7-SP26\DATxSLP"

# --- CẤU HÌNH ---
input_dir = r"D:\Study\7-SP26\DATxSLP\Data_after_preprocessing\test\id00005"
output_dir = r"D:\Study\7-SP26\DATxSLP\Data_after_cut\test_output"
english_dir = r"D:\Study\7-SP26\DATxSLP\Data_after_cut\file_english"

os.makedirs(output_dir, exist_ok=True)
os.makedirs(english_dir, exist_ok=True)

MODEL_SIZE = "large-v2" 
TARGET_SECONDS = 5.0
STRIDE_SECONDS = 1.0
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
COMPUTE_TYPE = "int8_float16" 


ALLOWED_MARGIN = 0.5  
MIN_DURATION_THRESHOLD = TARGET_SECONDS - ALLOWED_MARGIN

asr_options = {
    "n_mels": 128  
}

# 1. Load Model Whisper
print(f"Loading Whisper model: {MODEL_SIZE} on {DEVICE}...")
model = whisperx.load_model(MODEL_SIZE, device=DEVICE, compute_type=COMPUTE_TYPE, asr_options=asr_options)

# Dictionary lưu align models để không phải load lại nhiều lần
align_models = {}

for filename in os.listdir(input_dir):
    if not filename.lower().endswith((".wav", ".mp3")):
        continue

    input_path = os.path.join(input_dir, filename)
    base_name = os.path.splitext(filename)[0]
    print(f"\n--- Đang xử lý file: {filename} ---")
    
    try:
        # 2. Nhận diện sơ bộ để kiểm tra ngôn ngữ
        audio_data = whisperx.load_audio(input_path)
        result = model.transcribe(audio_data, batch_size=16)
        detected_lang = result.get("language", "unknown")
        print(f"Ngôn ngữ phát hiện: {detected_lang}")

        # 3. KIỂM TRA NẾU LÀ TIẾNG ANH (en) -> Di chuyển và bỏ qua
        if detected_lang == "en":
            print(f"-> Phát hiện tiếng Anh. Đang copy file vào: {english_dir}")
            shutil.copy(input_path, os.path.join(english_dir, filename))
            continue 

        # 4. Load Align Model
        if detected_lang not in align_models:
            print(f"Loading alignment model cho: {detected_lang}...")
            try:
                if detected_lang == "vi":
                    align_model_name = "nguyenvulebinh/wav2vec2-base-vi"
                    align_models[detected_lang] = whisperx.load_align_model(
                        language_code=detected_lang, 
                        device=DEVICE, 
                        model_name=align_model_name
                    )
                else:
                    align_models[detected_lang] = whisperx.load_align_model(
                        language_code=detected_lang, 
                        device=DEVICE
                    )
            except Exception as e:
                print(f"Không hỗ trợ Alignment cho ngôn ngữ '{detected_lang}'. Lỗi: {e}")
                continue

        model_a, metadata = align_models[detected_lang]
        
        # 5. Thực hiện Alignment để lấy thời gian từng từ chính xác
        result_aligned = whisperx.align(result["segments"], model_a, metadata, input_path, DEVICE)
        words = [w for w in result_aligned["word_segments"] if "start" in w and w["end"] is not None]

        if not words:
            print("Không tìm thấy mốc thời gian từ (word segments) để cắt.")
            continue

        # 6. Cắt Audio bằng Pydub
        audio = AudioSegment.from_file(input_path)
        total_duration = len(audio) / 1000.0
        current_mark = 0.0
        seg_index = 1

        while current_mark + TARGET_SECONDS <= total_duration:
            # Tìm từ bắt đầu >= current_mark
            start_word_idx = None
            for idx, w in enumerate(words):
                if w["start"] >= current_mark:
                    start_word_idx = idx
                    break
            
            if start_word_idx is None: break

            actual_start_time = words[start_word_idx]["start"]
            current_segment_words = []
            
            # Gom từ cho đến khi đủ TARGET_SECONDS
            for j in range(start_word_idx, len(words)):
                current_segment_words.append(words[j])
                if words[j]["end"] - actual_start_time >= TARGET_SECONDS:
                    break
            
            if not current_segment_words: break
                
            seg_duration = current_segment_words[-1]["end"] - actual_start_time
            
            # Kiểm tra xem đoạn cắt có >= (5.0 - 0.5) hay không
            if seg_duration >= MIN_DURATION_THRESHOLD:
                start_ms = int(actual_start_time * 1000)
                end_ms = int(current_segment_words[-1]["end"] * 1000)
                
                segment_audio = audio[start_ms:end_ms]
                out_filename = f"{base_name}_seg_{seg_index:03d}.wav"
                segment_audio.export(os.path.join(output_dir, out_filename), format="wav")
                seg_index += 1
            else:
                # print(f"   - Bỏ qua đoạn: {seg_duration:.2f}s (Ngắn hơn {MIN_DURATION_THRESHOLD}s)")
                pass

            current_mark += STRIDE_SECONDS

        print(f"-> Hoàn tất! Đã cắt được {seg_index-1} đoạn.")

    except Exception as e:
        print(f"Lỗi khi xử lý file {filename}: {e}")

print("\n>>> TẤT CẢ FILE ĐÃ ĐƯỢC XỬ LÝ XONG! <<<")